In [1]:
import pandas as pd
from datasets import load_dataset
import json
import os
import sys
import boto3
from botocore.client import Config
import faiss

## Prompt eng
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain.llms.bedrock import Bedrock
from langchain_aws import BedrockLLM
from langchain.vectorstores import faiss
from langchain_community.vectorstores import FAISS
from langchain.embeddings import BedrockEmbeddings

In [2]:
# Bedrock Client
session = boto3.Session(
    aws_access_key_id='ACCESS_KEY',
    aws_secret_access_key='SECRET_ACCESS_KEY',
    aws_session_token=boto3.client('sts').assume_role(
        RoleArn='arn:aws:iam::975050062872:role/ColabAccess',
        RoleSessionName='ColabSession'  # You can customize the session name
    )['Credentials']['SessionToken'],
)
region =session.region_name
print(region)

us-east-2


In [3]:
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 3})

bedrock_client = boto3.client("bedrock-runtime", region_name = region)

In [4]:
# Create/call LLM

def get_llama():
  # Create a Model
    llm = BedrockLLM(model_id='us.meta.llama3-2-1b-instruct-v1:0',
                client=bedrock_client,
                model_kwargs={'max_gen_len': 300,'temperature': 0.3})
    return llm

In [5]:
# Create Prompt template

prompt_temp = """
Human: You are a expert in movies and series on netflix. You know about the actors, directors, the geners of content. 
You are good with cross reference of content. (Example: Question- What is that movie about cricket in which the "Tare zameen par" actor acted? 
Answer-Lagan, because As Amir khan acted in "Tare Zameen Par" his cricket movie is "Lagan".
you should stick to the question asked about, keep answer close to facts. if the use askes to suggest contect in certain gener,
give top 5 contect in the relevent geners. (Example: Q-Suggest me super hero movies. A-Iron Man, Avergers, Batman) Don't add subcategories and give 5 
suggestion for each subcategoy. Just give 5 suggestion for each question. For each recomendation add 1-2 line description of content relevent to question.
if no conetxt is given in question suggest 5 content randomly. 
<context>
{context}
</context>
Question: {question}
Assistant:
"""

prompt = PromptTemplate(template=prompt_temp, input_variables=['context', 'question'])


In [6]:
# Create the embedding object again (used for loading)
bedrock_embedding = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0", 
    client=bedrock_client
)

# Load the saved FAISS index from local folder
vectorestore_fiass = FAISS.load_local("faiss_index", bedrock_embedding,allow_dangerous_deserialization=True)

/tmp/ipykernel_12374/3884776118.py:2: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  bedrock_embedding = BedrockEmbeddings(


In [7]:
# Get response from LLM

def get_response_llm(llm, vectorestore_fiass, query):
    retriever = vectorestore_fiass.as_retriever()

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={'prompt': prompt}
    )
    
    answer = qa({'query': query})  # Now it's fine again
    return answer['result']


In [8]:
# Load the LLM
llm = get_llama()

# Ask a query
query = input("Ask your movie-related question: ")

# Get response
response = get_response_llm(llm, vectorestore_fiass, query)  # if faiss_index is the path
print("Answer:", response)

Ask your movie-related question:  comedy movies


/tmp/ipykernel_12374/2375700674.py:14: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa({'query': query})  # Now it's fine again


Answer: You are a good match for this question. Here are 5 comedy movies that you might enjoy:

1. The Hangover (2009)
2. Superbad (2007)
3. Bridesmaids (2011)
4. The 40-Year-Old Virgin (2005)
5. Anchorman: The Legend of Ron Burgundy (2004)

These movies are all well-known comedies that have received critical acclaim and have been popular among audiences.

Would you like me to suggest more comedy movies or provide more information about these movies?
